In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('processed_data_davi.csv', dtype={
    "ICAO 24-bit code": 'str',
    "callsign": 'str',
    "origin country": 'str',
    "time at position": 'str',
    "time of last contact": 'str',
    "longitude": 'str',
    "latitude": 'str',
    "barometric altitude": 'float',
    "aircraft is grounded": 'bool',
    "velocity": 'float',
    "heading": 'float',
    "vertical rate": 'float',
    "geo_altitude": 'float',
    "squawk": 'float',
    "spi": 'bool',
    "latitude, longitude": 'str',
    "location": 'str',
    "country": 'str',
    "oblast": 'str',
}, date_parser = pd.to_datetime,
parse_dates=['time at position', 'time of last contact'])

In [3]:
df[df.country == 'Ukraine'].oblast.unique()

array([], dtype=object)

In [4]:
df.latitude = round(df.latitude.astype(float),4).astype(str)

In [5]:
df.longitude = round(df.longitude.astype(float),4).astype(str)

In [6]:
df.country = df.country[df.country.str[0] == " "].str[1:]

In [7]:
df['date'] = df['time at position'].apply(lambda x: str(x.date()))
df['flight-id'] = df['ICAO 24-bit code'] + df['callsign'] + df['date']

In [8]:
df.loc[df.country == "Hungary", "oblast"] = df[df.country == "Hungary"].location.str.split(",").apply(lambda x: [e for ind, e in enumerate(x[1:]) if ('járás' in x[ind])][0])

In [9]:
df.loc[df.country == "Poland", "oblast"] = df[df.country == "Poland"].location.str.split(",").apply(lambda x: x[-2] if 'Voivodeship' in x[-2] else x[-3])

In [10]:
df.loc[df.country == "Romania", "oblast"] = df[df.country == "Romania"].location.str.split(",").apply(lambda x: x[-3] if any(char.isdigit() for char in x[-2]) else x[-2])

In [11]:
df.loc[df.country == "Bulgaria", "oblast"] = df[df.country == "Bulgaria"].location.str.split(",").apply(lambda x: x[-3] if any(char.isdigit() for char in x[-2]) else x[-2])

In [12]:
df.loc[df.country == "Russia", "oblast"] = df[df.country == "Russia"].location.str.split(",").apply(lambda x: [e for e in x if ('Oblast' in e)][0] if len([e for e in x if ('Oblast' in e)]) > 0 else None)

In [13]:
df.loc[df.country == "Belarus", "oblast"] = df[df.country == "Belarus"].location.str.split(",").apply(lambda x: [e for e in x if ('Region' in e)][0] if len([e for e in x if ('Region' in e)]) > 0 else None)

In [14]:
df.loc[df.country == "Slovakia", "oblast"] = df[df.country == "Slovakia"].location.str.split(",").apply(lambda x: [e for e in x if ('Region' in e)][0])

In [15]:
df.loc[df.country == "Serbia", "oblast"] = df[df.country == "Serbia"].location.str.split(",").apply(lambda x: x[-3] if any(char.isdigit() for char in x[-2]) else x[-2])

In [16]:
df.oblast = df.oblast[df.oblast.str[0] == " "].str[1:]

In [17]:
df.groupby(['country','oblast'], dropna=False)['flight-id'].nunique().to_csv("oblasts.csv")

In [18]:
df['latitude, longitude'] = df[['latitude', 'longitude']].values.tolist()

In [24]:
gbdf = df.sort_values(by='time at position').groupby(['flight-id','origin country','date'], dropna=False).agg({
    "barometric altitude": np.max,
    "velocity": np.max,
    "vertical rate": np.max,
    "oblast": lambda x: set(x),
    "country": lambda x: set(x),
    "latitude, longitude": lambda x: list(x)
})

In [ ]:
df[df.country == 'Ukraine'].to_csv('new_data_davi.csv', index=False) 

In [25]:
gbdf.to_csv("flights_separate.csv")

In [23]:
df.sort_values(by='date').head()

,ICAO 24-bit code,callsign,origin country,time at position,time of last contact,longitude,latitude,barometric altitude,aircraft is grounded,velocity,...,vertical rate,geo_altitude,squawk,spi,"latitude, longitude",location,country,oblast,date,flight-id
0,471f8c,WZZ1550,Hungary,2022-02-22 15:28:28,2022-02-22 15:28:34,21.2426,45.9153,10668.0,False,231.00,...,-0.33,10553.70,6405.0,False,"[45.9153, 21.2426]","DN69, Orțișoara, Timiș, 307307, Romania",Romania,Timiș,2022-02-22,471f8cWZZ15502022-02-22
21511,4baa72,THY4TA,Turkey,2022-02-22 19:39:18,2022-02-22 19:39:18,28.3571,43.4474,10058.4,False,208.69,...,-0.33,9974.58,4531.0,False,"[43.4474, 28.3571]","Balgarevo, Kavarna, Dobrich, Bulgaria",Bulgaria,Dobrich,2022-02-22,4baa72THY4TA2022-02-22
21510,4baa66,THY52D,Turkey,2022-02-22 19:39:17,2022-02-22 19:39:18,26.5575,43.9758,9448.8,False,223.83,...,0.33,9357.36,7624.0,False,"[43.9758, 26.5575]","Tutrakan, Silistra, 7600, Bulgaria",Bulgaria,Silistra,2022-02-22,4baa66THY52D2022-02-22
21509,4baa90,THY45K,Turkey,2022-02-22 19:39:17,2022-02-22 19:39:17,25.5156,42.8842,10668.0,False,232.10,...,0.00,10561.32,7634.0,False,"[42.8842, 25.5156]","Staynovtsi, Triavna, Gabrovo, 5350, Bulgaria",Bulgaria,Gabrovo,2022-02-22,4baa90THY45K2022-02-22
21508,4bb141,THY10,Turkey,2022-02-22 19:39:17,2022-02-22 19:39:18,24.105,43.8004,10668.0,False,251.22,...,0.00,10553.70,2243.0,False,"[43.8004, 24.105]","Strada Unirii, Dabuleni, Dolj, 207220, Romania",Romania,Dolj,2022-02-22,4bb141THY102022-02-22
